In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import os
import sys
from pathlib import Path
from src.story import create_html_challenges
chat_app_dir = Path().absolute().parent / "chat-app"
assert chat_app_dir.exists()
if (str(chat_app_dir)) not in sys.path:
    sys.path.append(str(chat_app_dir))
from src_code.utils.build import create_chat_interface


setting voice override: sv-SE-Wavenet-F
setting voice override: sv-SE-Wavenet-G
FFmpeg path added to system PATH: C:\Program Files\ffmpeg-7.0-essentials_build\bin


In [4]:
notebook_dir = Path().absolute()  # This gives src/notebooks
phrase_dir = notebook_dir.parent / "data" / "phrases" #where we store text files of phrases
story_dir = notebook_dir.parent / "outputs" / "stories" # where we store our stories
chat_dir = notebook_dir.parent / "chat"

In [5]:

# Example challenges for testing
test_challenges = [
    {
        "challenge_description": "You are at a coffee shop, find out the server's name.",
        "llm_prompt": """You are a barista at a coffee shop. Your name is Willow.""",
        "answer": "The person's name is Willow"
    },
    {
        "challenge_description": "Order a coffee in Swedish and ask how much it costs",
        "llm_prompt": """You are a role playing Swedish café worker for language learning. The learners has a challenge to order a coffee and find the price (they cost 10 kronor).
        Respond naturally in Swedish; guiding the learner (using swedish) if the make a mistake.""",
        "answer": "The coffee costs 10 kronor (price in Swedish: 'Det kostar tio kronor')"
    },
    {
    "challenge_description": "Order 2 beers in Russian and find out what the waitress thinks of your friend",
    "llm_prompt": """IMPORTANT: Respond ONLY in Russian, no English.
    
    You are a waitress at a Russian bar. When customers order drinks, respond only in Russian. If they ask about their friend, say "очень красивый" (very handsome). If they make mistakes, guide them in Russian.
    
    DO NOT use any English - maintain Russian immersion.""",
    "answer": "The waitress thinks the friend is very handsome (in Russian: 'очень красивый')"
}
]


# Create the HTML file with test challenges
create_html_challenges(
    challenges=test_challenges,
    output_dir=chat_dir,
    title="DuoLaingo Conversation Practice",
    language="English and Swedish"
)

HTML challenges created at: y:\Python Scripts\audio-language-trainer\chat\duolaingo_conversation_practice.html
